# 1. Set up environment

## 1.1 Install a Virtual env with all dependencies

### 1.1.1 UV Based Environment Creation
- Running below cell  requires uv to be installed on your machine. 
- You can install from https://docs.astral.sh/uv/pip/environments/
- If you dont want UV please use pip based install

In [ ]:
%%bash
python -m pip install uv
uv venv ray_jup_env
source ray_jup_env/bin/activate

uv pip install ray[serve] #this is how you install ray_serve python package
uv pip install ipykernel nbconvert ipywidgets #these are required to attach created environment in notebook
python -m ipykernel install --user --name=ray_jup_env


### 1.1.2 PIP Based Environment Creation
 - Uncomment below and run if you do want to not use above uv base install

In [ ]:
# %%bash
# python -m pip install --user virtualenv
# python -m virtualenv ray_jup_env
# source ray_jup_env/bin/activate
# python -m pip install ray[serve] #this is how you install ray_serve python package
# python -m pip install nest-asyncio #this is required to run a FastAPI app in non-blocking mode from a jupyter notebook
# python -m pip install ipykernel nbconvert ipywidgets #these are required to attach created environment in notebook
# python -m ipykernel install --user --name=ray_jup_env

## 1.2 Activate the Kernel
- refresh the browser
- activate the _ray_jup_env_ kernel

# 2. Simple FastAPI Endpoint

## 2.1 Create a simple FastAPI endpoint

In [ ]:
from fastapi import FastAPI
import os

app = FastAPI()


@app.get("/hello")
def hello():
    return {"message": "Hello, World"}

## 2.2 Serve the Fast API end point
  - I am using an asyncio loop to serve the app in an async way from notebook
  - if you are running this inside a container, you should now see at http://localhost:8001/docs

In [ ]:
import asyncio
import uvicorn

if __name__ == "__main__":
    config = uvicorn.Config(app, host="0.0.0.0", port=8002)
    server = uvicorn.Server(config)
    loop = asyncio.get_running_loop()
    loop.create_task(server.serve())

print("If you are running this inside a container, you should now see at http://localhost:8002/hello")

# 3. Deploy Fast API app Using Ray Serve

## 3.1 Start a Ray Cluster

In [ ]:
import ray
ray.init(num_cpus=8,dashboard_host="0.0.0.0")

## 3.2 Convert the FastAPI app to a RayDeployment

In [ ]:
#Need to convert FastAPI app to a ray deployment actor

from ray import serve

@serve.deployment
@serve.ingress(app)
class RayApp:
    pass


rayapp = RayApp.bind()

## 3.3 Another ray app with parmeterized information

In [ ]:
from ray import serve


@serve.deployment(num_replicas=2, ray_actor_options={"num_cpus": 1})
@serve.ingress(app)
class RayAppParamaeterized:
    pass


rayappparameterized = RayAppParamaeterized.bind()

## 3.4 Another ray app with parmeterized information requesting more resource than avaialbale

In [ ]:
#Below is a NegativExample
from ray import serve

@serve.deployment(num_replicas=6, ray_actor_options={"num_cpus": 1})
@serve.ingress(app)
class RayAppParamaeterizedFail:
    pass


rayappparameterizedfail = RayAppParamaeterizedFail.bind()

## 3.5 Deploy on a  ray serve cluster

If you are running this inside a container, you should now see at http://localhost:8265

### 3.5.1 Start Serve Instance on Ray Cluster

In [ ]:
serve.start(http_options={"host":"0.0.0.0"})

### 3.5.2 Deploy above Ray Wrapped Fast API apps on Ray Serve

In [ ]:
serve.run(rayapp, name="fastapiappponray")
print("Served app should be visible at http://localhost:8000/hello")

In [ ]:
serve.run(rayappparameterized, 
          name="fastapiappponrayparameterized", 
          route_prefix="/parameterized_app")
print("Served app should be visible at http://localhost:8000/parameterized_app/hello")

In [ ]:
serve.run(rayappparameterizedfail, 
          name="rayappparameterizedfail", 
          # route_prefix="/parameterized_app_fail")
print("Served app should be visible at http://localhost:8000/parameterized_app_fail/hello, however all replicas would not come up")

In [ ]:
serve.shutdown()

### 3.5.3  ShutDown Ray Cluster

In [ ]:
ray.shutdown()

In [ ]:
# %%bash
# be carefu with this
# rm -rf ray_jup_env